Ok 
i am mixing here

RAKE
+ TFIDF ngram(1-3)
and group together to make a tag.
These tags we will see the ranking soon

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import string
import nltk
import operator

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix


def dict_to_df(d):
    df = pd.DataFrame()
    df['word'] = d.keys()
    df['count'] = d.values()
    return df
RES_DIR = "../input/"
def load_train_data():
    categories = ['cooking', 'robotics', 'travel', 'crypto', 'diy', 'biology']
    train_data = []
    for cat in categories:
        data = pd.read_csv("{}{}.csv".format(RES_DIR, cat), usecols=['id', 'title','content' ,'tags'])
        data['category'] = cat
        train_data.append(data)
    return pd.concat(train_data)
train_data = load_train_data()
#import the test data

#print(train_data.head())

#import test data in same format
testdoc = pd.read_csv("../input/test.csv")
testdoc['tags'] = ''
testdoc['category'] = 'physics'
print(testdoc.head())
#train_data_tot=train_data.append(testdoc)
# Implementation of RAKE - Rapid Automtic Keyword Exraction algorithm


uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""
    
def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False


def load_stop_words(stop_word_file):
    """
    Utility function to load stop words from a file and return as a list of words
    @param stop_word_file Path and file name of a file containing stop words.
    @return list A list of stop words.
    """
    stop_words = []
    for line in open(stop_word_file):
        if line.strip()[0:1] != "#":
            for word in line.split():  # in case more than one per line
                stop_words.append(word)
    return stop_words


def separate_words(text, min_word_return_size):
    """
    Utility function to return a list of all words that are have a length greater than a specified number of characters.
    @param text The text that must be split in to words.
    @param min_word_return_size The minimum no of characters a word must have to be included.
    """
    splitter = re.compile('[^a-zA-Z0-9_\\+\\-/]')
    words = []
    for single_word in splitter.split(text):
        current_word = single_word.strip().lower()
        #leave numbers in phrase, but don't count as words, since they tend to invalidate scores of their phrases
        if len(current_word) > min_word_return_size and current_word != '' and not is_number(current_word):
            words.append(current_word)
    return words


def split_sentences(text):
    """
    Utility function to return a list of sentences.
    @param text The text that must be split in to sentences.
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    return sentences


def build_stop_word_regex(stop_word_file_path):
    stop_word_list = load_stop_words(stop_word_file_path)
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = r'\b' + word + r'(?![\w-])'  # added look ahead for hyphen
        stop_word_regex_list.append(word_regex)
    stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)
    return stop_word_pattern

def build_stop_word_(stop_word_file_path):
    stop_word_list = stop_word_file_path
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = r'\b' + word + r'(?![\w-])'  # added look ahead for hyphen
        stop_word_regex_list.append(word_regex)
    stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)
    return stop_word_pattern

def generate_candidate_keywords(sentence_list, stopword_pattern):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stopword_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != "":
                phrase_list.append(phrase)
    return phrase_list


def calculate_word_scores(phraseList):
    word_frequency = {}
    word_degree = {}
    for phrase in phraseList:
        word_list = separate_words(phrase, 0)
        word_list_length = len(word_list)
        word_list_degree = word_list_length - 1
        if word_list_degree > 3: word_list_degree = 3 #exp.
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1
            word_degree.setdefault(word, 0)
            #word_degree[word] += word_list_degree  #orig.
            word_degree[word] += 1/(word_list_length*1.0) #exp.
    for item in word_frequency:
        word_degree[item] = word_degree[item] + word_frequency[item]

    # Calculate Word scores = deg(w)/frew(w)
    word_score = {}
    for item in word_frequency:
        word_score.setdefault(item, 0)
        # word_score[item] = word_degree[item] / (word_frequency[item] * 1.0)  #orig.
        word_score[item] = word_frequency[item]/(word_degree[item] * 1.0) #exp.
    return word_score


def generate_candidate_keyword_scores(phrase_list, word_score):
    keyword_candidates = {}
    for phrase in phrase_list:
        keyword_candidates.setdefault(phrase, 2)
        word_list = separate_words(phrase, 2)
        candidate_score = 0
        for word in word_list:
            candidate_score += word_score[word]
        keyword_candidates[phrase] = candidate_score
    return keyword_candidates


class Rake(object):
    def __init__(self, stop_words_path):
        self.stop_words_path = stop_words_path
        self.__stop_words_pattern = build_stop_word_regex(stop_words_path)

    def run(self, text):
        sentence_list = split_sentences(text)
        phrase_list = generate_candidate_keywords(sentence_list, self.__stop_words_pattern)
        word_scores = calculate_word_scores(phrase_list)
        keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores)
        sorted_keywords = sorted(keyword_candidates.items(), key=operator.itemgetter(1), reverse=True)
        return sorted_keywords

debug=True
for index, row in testdoc.sample(2).iterrows():
    #print(row["title"], row["content"]    )
    sentenceList = split_sentences(row['title']+' '+row['content'])
    #stoppath = "FoxStoplist.txt" #Fox stoplist contains "numbers", so it will not find "natural numbers" like in Table 1.1
    stoppath = "SmartStoplist.txt"  #SMART stoplist misses some of the lower-scoring keywords in Figure 1.5, which means that the top 1/3 cuts off one of the 4.0 score words in Table 1.1
    #stopwordpattern = build_stop_word_regex(stoppath)
    stopwordpattern= """re.compile(r"\ba(?![\w-])|\ba's(?![\w-])|\bable(?![\w-])|\babout(?![\w-])|\babove(?![\w-])|\baccording(?![\w-])|\baccordingly(?![\w-])|\bacross(?![\w-])|\bactually(?![\w-])|\bafter(?![\w-])|\bafterwards(?![\w-])|\bagain(?![\w-])|\bagainst(?![\w-])|\bain't(?![\w-])|\ball(?![\w-])|\ballow(?![\w-])|\ballows(?![\w-])|\balmost(?![\w-])|\balone(?![\w-])|\balong(?![\w-])|\balready(?![\w-])|\balso(?![\w-])|\balthough(?![\w-])|\balways(?![\w-])|\bam(?![\w-])|\bamong(?![\w-])|\bamongst(?![\w-])|\ban(?![\w-])|\band(?![\w-])|\banother(?![\w-])|\bany(?![\w-])|\banybody(?![\w-])|\banyhow(?![\w-])|\banyone(?![\w-])|\banything(?![\w-])|\banyway(?![\w-])|\banyways(?![\w-])|\banywhere(?![\w-])|\bapart(?![\w-])|\bappear(?![\w-])|\bappreciate(?![\w-])|\bappropriate(?![\w-])|\bare(?![\w-])|\baren't(?![\w-])|\baround(?![\w-])|\bas(?![\w-])|\baside(?![\w-])|\bask(?![\w-])|\basking(?![\w-])|\bassociated(?![\w-])|\bat(?![\w-])|\bavailable(?![\w-])|\baway(?![\w-])|\bawfully(?![\w-])|\bb(?![\w-])|\bbe(?![\w-])|\bbecame(?![\w-])|\bbecause(?![\w-])|\bbecome(?![\w-])|\bbecomes(?![\w-])|\bbecoming(?![\w-])|\bbeen(?![\w-])|\bbefore(?![\w-])|\bbeforehand(?![\w-])|\bbehind(?![\w-])|\bbeing(?![\w-])|\bbelieve(?![\w-])|\bbelow(?![\w-])|\bbeside(?![\w-])|\bbesides(?![\w-])|\bbest(?![\w-])|\bbetter(?![\w-])|\bbetween(?![\w-])|\bbeyond(?![\w-])|\bboth(?![\w-])|\bbrief(?![\w-])|\bbut(?![\w-])|\bby(?![\w-])|\bc(?![\w-])|\bc'mon(?![\w-])|\bc's(?![\w-])|\bcame(?![\w-])|\bcan(?![\w-])|\bcan't(?![\w-])|\bcannot(?![\w-])|\bcant(?![\w-])|\bcause(?![\w-])|\bcauses(?![\w-])|\bcertain(?![\w-])|\bcertainly(?![\w-])|\bchanges(?![\w-])|\bclearly(?![\w-])|\bco(?![\w-])|\bcom(?![\w-])|\bcome(?![\w-])|\bcomes(?![\w-])|\bconcerning(?![\w-])|\bconsequently(?![\w-])|\bconsider(?![\w-])|\bconsidering(?![\w-])|\bcontain(?![\w-])|\bcontaining(?![\w-])|\bcontains(?![\w-])|\bcorresponding(?![\w-])|\bcould(?![\w-])|\bcouldn't(?![\w-])|\bcourse(?![\w-])|\bcurrently(?![\w-])|\bd(?![\w-])|\bdefinitely(?![\w-])|\bdescribed(?![\w-])|\bdespite(?![\w-])|\bdid(?![\w-])|\bdidn't(?![\w-])|\bdifferent(?![\w-])|\bdo(?![\w-])|\bdoes(?![\w-])|\bdoesn't(?![\w-])|\bdoing(?![\w-])|\bdon't(?![\w-])|\bdone(?![\w-])|\bdown(?![\w-])|\bdownwards(?![\w-])|\bduring(?![\w-])|\be(?![\w-])|\beach(?![\w-])|\bedu(?![\w-])|\beg(?![\w-])|\beight(?![\w-])|\beither(?![\w-])|\belse(?![\w-])|\belsewhere(?![\w-])|\benough(?![\w-])|\bentirely(?![\w-])|\bespecially(?![\w-])|\bet(?![\w-])|\betc(?![\w-])|\beven(?![\w-])|\bever(?![\w-])|\bevery(?![\w-])|\beverybody(?![\w-])|\beveryone(?![\w-])|\beverything(?![\w-])|\beverywhere(?![\w-])|\bex(?![\w-])|\bexactly(?![\w-])|\bexample(?![\w-])|\bexcept(?![\w-])|\bf(?![\w-])|\bfar(?![\w-])|\bfew(?![\w-])|\bfifth(?![\w-])|\bfirst(?![\w-])|\bfive(?![\w-])|\bfollowed(?![\w-])|\bfollowing(?![\w-])|\bfollows(?![\w-])|\bfor(?![\w-])|\bformer(?![\w-])|\bformerly(?![\w-])|\bforth(?![\w-])|\bfour(?![\w-])|\bfrom(?![\w-])|\bfurther(?![\w-])|\bfurthermore(?![\w-])|\bg(?![\w-])|\bget(?![\w-])|\bgets(?![\w-])|\bgetting(?![\w-])|\bgiven(?![\w-])|\bgives(?![\w-])|\bgo(?![\w-])|\bgoes(?![\w-])|\bgoing(?![\w-])|\bgone(?![\w-])|\bgot(?![\w-])|\bgotten(?![\w-])|\bgreetings(?![\w-])|\bh(?![\w-])|\bhad(?![\w-])|\bhadn't(?![\w-])|\bhappens(?![\w-])|\bhardly(?![\w-])|\bhas(?![\w-])|\bhasn't(?![\w-])|\bhave(?![\w-])|\bhaven't(?![\w-])|\bhaving(?![\w-])|\bhe(?![\w-])|\bhe's(?![\w-])|\bhello(?![\w-])|\bhelp(?![\w-])|\bhence(?![\w-])|\bher(?![\w-])|\bhere(?![\w-])|\bhere's(?![\w-])|\bhereafter(?![\w-])|\bhereby(?![\w-])|\bherein(?![\w-])|\bhereupon(?![\w-])|\bhers(?![\w-])|\bherself(?![\w-])|\bhi(?![\w-])|\bhim(?![\w-])|\bhimself(?![\w-])|\bhis(?![\w-])|\bhither(?![\w-])|\bhopefully(?![\w-])|\bhow(?![\w-])|\bhowbeit(?![\w-])|\bhowever(?![\w-])|\bi(?![\w-])|\bi'd(?![\w-])|\bi'll(?![\w-])|\bi'm(?![\w-])|\bi've(?![\w-])|\bie(?![\w-])|\bif(?![\w-])|\bignored(?![\w-])|\bimmediate(?![\w-])|\bin(?![\w-])|\binasmuch(?![\w-])|\binc(?![\w-])|\bindeed(?![\w-])|\bindicate(?![\w-])|\bindicated(?![\w-])|\bindicates(?![\w-])|\binner(?![\w-])|\binsofar(?![\w-])|\binstead(?![\w-])|\binto(?![\w-])|\binward(?![\w-])|\bis(?![\w-])|\bisn't(?![\w-])|\bit(?![\w-])|\bit'd(?![\w-])|\bit'll(?![\w-])|\bit's(?![\w-])|\bits(?![\w-])|\bitself(?![\w-])|\bj(?![\w-])|\bjust(?![\w-])|\bk(?![\w-])|\bkeep(?![\w-])|\bkeeps(?![\w-])|\bkept(?![\w-])|\bknow(?![\w-])|\bknows(?![\w-])|\bknown(?![\w-])|\bl(?![\w-])|\blast(?![\w-])|\blately(?![\w-])|\blater(?![\w-])|\blatter(?![\w-])|\blatterly(?![\w-])|\bleast(?![\w-])|\bless(?![\w-])|\blest(?![\w-])|\blet(?![\w-])|\blet's(?![\w-])|\blike(?![\w-])|\bliked(?![\w-])|\blikely(?![\w-])|\blittle(?![\w-])|\blook(?![\w-])|\blooking(?![\w-])|\blooks(?![\w-])|\bltd(?![\w-])|\bm(?![\w-])|\bmainly(?![\w-])|\bmany(?![\w-])|\bmay(?![\w-])|\bmaybe(?![\w-])|\bme(?![\w-])|\bmean(?![\w-])|\bmeanwhile(?![\w-])|\bmerely(?![\w-])|\bmight(?![\w-])|\bmore(?![\w-])|\bmoreover(?![\w-])|\bmost(?![\w-])|\bmostly(?![\w-])|\bmuch(?![\w-])|\bmust(?![\w-])|\bmy(?![\w-])|\bmyself(?![\w-])|\bn(?![\w-])|\bname(?![\w-])|\bnamely(?![\w-])|\bnd(?![\w-])|\bnear(?![\w-])|\bnearly(?![\w-])|\bnecessary(?![\w-])|\bneed(?![\w-])|\bneeds(?![\w-])|\bneither(?![\w-])|\bnever(?![\w-])|\bnevertheless(?![\w-])|\bnew(?![\w-])|\bnext(?![\w-])|\bnine(?![\w-])|\bno(?![\w-])|\bnobody(?![\w-])|\bnon(?![\w-])|\bnone(?![\w-])|\bnoone(?![\w-])|\bnor(?![\w-])|\bnormally(?![\w-])|\bnot(?![\w-])|\bnothing(?![\w-])|\bnovel(?![\w-])|\bnow(?![\w-])|\bnowhere(?![\w-])|\bo(?![\w-])|\bobviously(?![\w-])|\bof(?![\w-])|\boff(?![\w-])|\boften(?![\w-])|\boh(?![\w-])|\bok(?![\w-])|\bokay(?![\w-])|\bold(?![\w-])|\bon(?![\w-])|\bonce(?![\w-])|\bone(?![\w-])|\bones(?![\w-])|\bonly(?![\w-])|\bonto(?![\w-])|\bor(?![\w-])|\bother(?![\w-])|\bothers(?![\w-])|\botherwise(?![\w-])|\bought(?![\w-])|\bour(?![\w-])|\bours(?![\w-])|\bourselves(?![\w-])|\bout(?![\w-])|\boutside(?![\w-])|\bover(?![\w-])|\boverall(?![\w-])|\bown(?![\w-])|\bp(?![\w-])|\bparticular(?![\w-])|\bparticularly(?![\w-])|\bper(?![\w-])|\bperhaps(?![\w-])|\bplaced(?![\w-])|\bplease(?![\w-])|\bplus(?![\w-])|\bpossible(?![\w-])|\bpresumably(?![\w-])|\bprobably(?![\w-])|\bprovides(?![\w-])|\bq(?![\w-])|\bque(?![\w-])|\bquite(?![\w-])|\bqv(?![\w-])|\br(?![\w-])|\brather(?![\w-])|\brd(?![\w-])|\bre(?![\w-])|\breally(?![\w-])|\breasonably(?![\w-])|\bregarding(?![\w-])|\bregardless(?![\w-])|\bregards(?![\w-])|\brelatively(?![\w-])|\brespectively(?![\w-])|\bright(?![\w-])|\bs(?![\w-])|\bsaid(?![\w-])|\bsame(?![\w-])|\bsaw(?![\w-])|\bsay(?![\w-])|\bsaying(?![\w-])|\bsays(?![\w-])|\bsecond(?![\w-])|\bsecondly(?![\w-])|\bsee(?![\w-])|\bseeing(?![\w-])|\bseem(?![\w-])|\bseemed(?![\w-])|\bseeming(?![\w-])|\bseems(?![\w-])|\bseen(?![\w-])|\bself(?![\w-])|\bselves(?![\w-])|\bsensible(?![\w-])|\bsent(?![\w-])|\bserious(?![\w-])|\bseriously(?![\w-])|\bseven(?![\w-])|\bseveral(?![\w-])|\bshall(?![\w-])|\bshe(?![\w-])|\bshould(?![\w-])|\bshouldn't(?![\w-])|\bsince(?![\w-])|\bsix(?![\w-])|\bso(?![\w-])|\bsome(?![\w-])|\bsomebody(?![\w-])|\bsomehow(?![\w-])|\bsomeone(?![\w-])|\bsomething(?![\w-])|\bsometime(?![\w-])|\bsometimes(?![\w-])|\bsomewhat(?![\w-])|\bsomewhere(?![\w-])|\bsoon(?![\w-])|\bsorry(?![\w-])|\bspecified(?![\w-])|\bspecify(?![\w-])|\bspecifying(?![\w-])|\bstill(?![\w-])|\bsub(?![\w-])|\bsuch(?![\w-])|\bsup(?![\w-])|\bsure(?![\w-])|\bt(?![\w-])|\bt's(?![\w-])|\btake(?![\w-])|\btaken(?![\w-])|\btell(?![\w-])|\btends(?![\w-])|\bth(?![\w-])|\bthan(?![\w-])|\bthank(?![\w-])|\bthanks(?![\w-])|\bthanx(?![\w-])|\bthat(?![\w-])|\bthat's(?![\w-])|\bthats(?![\w-])|\bthe(?![\w-])|\btheir(?![\w-])|\btheirs(?![\w-])|\bthem(?![\w-])|\bthemselves(?![\w-])|\bthen(?![\w-])|\bthence(?![\w-])|\bthere(?![\w-])|\bthere's(?![\w-])|\bthereafter(?![\w-])|\bthereby(?![\w-])|\btherefore(?![\w-])|\btherein(?![\w-])|\btheres(?![\w-])|\bthereupon(?![\w-])|\bthese(?![\w-])|\bthey(?![\w-])|\bthey'd(?![\w-])|\bthey'll(?![\w-])|\bthey're(?![\w-])|\bthey've(?![\w-])|\bthink(?![\w-])|\bthird(?![\w-])|\bthis(?![\w-])|\bthorough(?![\w-])|\bthoroughly(?![\w-])|\bthose(?![\w-])|\bthough(?![\w-])|\bthree(?![\w-])|\bthrough(?![\w-])|\bthroughout(?![\w-])|\bthru(?![\w-])|\bthus(?![\w-])|\bto(?![\w-])|\btogether(?![\w-])|\btoo(?![\w-])|\btook(?![\w-])|\btoward(?![\w-])|\btowards(?![\w-])|\btried(?![\w-])|\btries(?![\w-])|\btruly(?![\w-])|\btry(?![\w-])|\btrying(?![\w-])|\btwice(?![\w-])|\btwo(?![\w-])|\bu(?![\w-])|\bun(?![\w-])|\bunder(?![\w-])|\bunfortunately(?![\w-])|\bunless(?![\w-])|\bunlikely(?![\w-])|\buntil(?![\w-])|\bunto(?![\w-])|\bup(?![\w-])|\bupon(?![\w-])|\bus(?![\w-])|\buse(?![\w-])|\bused(?![\w-])|\buseful(?![\w-])|\buses(?![\w-])|\busing(?![\w-])|\busually(?![\w-])|\buucp(?![\w-])|\bv(?![\w-])|\bvalue(?![\w-])|\bvarious(?![\w-])|\bvery(?![\w-])|\bvia(?![\w-])|\bviz(?![\w-])|\bvs(?![\w-])|\bw(?![\w-])|\bwant(?![\w-])|\bwants(?![\w-])|\bwas(?![\w-])|\bwasn't(?![\w-])|\bway(?![\w-])|\bwe(?![\w-])|\bwe'd(?![\w-])|\bwe'll(?![\w-])|\bwe're(?![\w-])|\bwe've(?![\w-])|\bwelcome(?![\w-])|\bwell(?![\w-])|\bwent(?![\w-])|\bwere(?![\w-])|\bweren't(?![\w-])|\bwhat(?![\w-])|\bwhat's(?![\w-])|\bwhatever(?![\w-])|\bwhen(?![\w-])|\bwhence(?![\w-])|\bwhenever(?![\w-])|\bwhere(?![\w-])|\bwhere's(?![\w-])|\bwhereafter(?![\w-])|\bwhereas(?![\w-])|\bwhereby(?![\w-])|\bwherein(?![\w-])|\bwhereupon(?![\w-])|\bwherever(?![\w-])|\bwhether(?![\w-])|\bwhich(?![\w-])|\bwhile(?![\w-])|\bwhither(?![\w-])|\bwho(?![\w-])|\bwho's(?![\w-])|\bwhoever(?![\w-])|\bwhole(?![\w-])|\bwhom(?![\w-])|\bwhose(?![\w-])|\bwhy(?![\w-])|\bwill(?![\w-])|\bwilling(?![\w-])|\bwish(?![\w-])|\bwith(?![\w-])|\bwithin(?![\w-])|\bwithout(?![\w-])|\bwon't(?![\w-])|\bwonder(?![\w-])|\bwould(?![\w-])|\bwould(?![\w-])|\bwouldn't(?![\w-])|\bx(?![\w-])|\by(?![\w-])|\byes(?![\w-])|\byet(?![\w-])|\byou(?![\w-])|\byou'd(?![\w-])|\byou'll(?![\w-])|\byou're(?![\w-])|\byou've(?![\w-])|\byour(?![\w-])|\byours(?![\w-])|\byourself(?![\w-])|\byourselves(?![\w-])|\bz(?![\w-])|\bzero(?![\w-])",
re.IGNORECASE|re.UNICODE)"""
    # generate candidate keywords
    phraseList = generate_candidate_keywords(sentenceList, stopwordpattern)
    if debug: print('_________Phrases',phraseList)
    # calculate individual word scores
    wordscores = calculate_word_scores(phraseList)
    if debug: print('_________wordscores',wordscores)
    # generate candidate keyword scores
    keywordcandidates = generate_candidate_keyword_scores(phraseList, wordscores)
    if debug: print('_________keywcand',keywordcandidates)

    sortedKeywords = sorted(keywordcandidates.items(), key=operator.itemgetter(1), reverse=True)
    if debug: print('_________sorkeyw',sortedKeywords)

    totalKeywords = len(sortedKeywords)
    if debug: print('_________totalkeyw',totalKeywords)
    print (sortedKeywords[0:int(totalKeywords / 3)])

    #rake = Rake("E:/input/SmartStoplist.txt")
  

In [ ]:
# Implementation of RAKE - Rapid Automtic Keyword Exraction algorithm
# as described in:
# Rose, S., D. Engel, N. Cramer, and W. Cowley (2010). 
# Automatic keyword extraction from indi-vidual documents. 
# In M. W. Berry and J. Kogan (Eds.), Text Mining: Applications and Theory.unknown: John Wiley and Sons, Ltd.
#
# NOTE: The original code (from https://github.com/aneesha/RAKE)
# has been extended by a_medelyan (zelandiya)
# with a set of heuristics to decide whether a phrase is an acceptable candidate
# as well as the ability to set frequency and phrase length parameters
# important when dealing with longer documents

from __future__ import absolute_import
from __future__ import print_function
import re
import operator
import six
from six.moves import range

debug = True
test = True


uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""
    

def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False


def load_stop_words(stop_word_file):
    """
    Utility function to load stop words from a file and return as a list of words
    @param stop_word_file Path and file name of a file containing stop words.
    @return list A list of stop words.
    """
    stop_words = []
    for line in open(stop_word_file):
        if line.strip()[0:1] != "#":
            for word in line.split():  # in case more than one per line
                stop_words.append(word)
    return stop_words


def separate_words(text, min_word_return_size):
    """
    Utility function to return a list of all words that are have a length greater than a specified number of characters.
    @param text The text that must be split in to words.
    @param min_word_return_size The minimum no of characters a word must have to be included.
    """
    splitter = re.compile('[^a-zA-Z0-9_\\+\\-/]')
    words = []
    for single_word in splitter.split(text):
        current_word = single_word.strip().lower()
        #leave numbers in phrase, but don't count as words, since they tend to invalidate scores of their phrases
        if len(current_word) > min_word_return_size and current_word != '' and not is_number(current_word):
            words.append(current_word)
    return words


def split_sentences(text):
    """
    Utility function to return a list of sentences.
    @param text The text that must be split in to sentences.
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    return sentences


def build_stop_word_regex(stop_word_file_path):
    stop_word_list = load_stop_words(stop_word_file_path)
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = '\\b' + word + '\\b'
        stop_word_regex_list.append(word_regex)
    stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)
    return stop_word_pattern


def generate_candidate_keywords(sentence_list, stopword_pattern, min_char_length=1, max_words_length=5):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stopword_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != "" and is_acceptable(phrase, min_char_length, max_words_length):
                phrase_list.append(phrase)
    return phrase_list


def is_acceptable(phrase, min_char_length, max_words_length):

    # a phrase must have a min length in characters
    if len(phrase) < min_char_length:
        return 0

    # a phrase must have a max number of words
    words = phrase.split()
    if len(words) > max_words_length:
        return 0

    digits = 0
    alpha = 0
    for i in range(0, len(phrase)):
        if phrase[i].isdigit():
            digits += 1
        elif phrase[i].isalpha():
            alpha += 1

    # a phrase must have at least one alpha character
    if alpha == 0:
        return 0

    # a phrase must have more alpha than digits characters
    if digits > alpha:
        return 0
    return 1


def calculate_word_scores(phraseList):
    word_frequency = {}
    word_degree = {}
    for phrase in phraseList:
        word_list = separate_words(phrase, 0)
        word_list_length = len(word_list)
        word_list_degree = word_list_length - 1
        #if word_list_degree > 3: word_list_degree = 3 #exp.
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1
            word_degree.setdefault(word, 0)
            word_degree[word] += word_list_degree  #orig.
            #word_degree[word] += 1/(word_list_length*1.0) #exp.
    for item in word_frequency:
        word_degree[item] = word_degree[item] + word_frequency[item]

    # Calculate Word scores = deg(w)/frew(w)
    word_score = {}
    for item in word_frequency:
        word_score.setdefault(item, 0)
        word_score[item] = word_degree[item] / (word_frequency[item] * 1.0)  #orig.
    #word_score[item] = word_frequency[item]/(word_degree[item] * 1.0) #exp.
    return word_score


def generate_candidate_keyword_scores(phrase_list, word_score, min_keyword_frequency=1):
    keyword_candidates = {}

    for phrase in phrase_list:
        if min_keyword_frequency > 1:
            if phrase_list.count(phrase) < min_keyword_frequency:
                continue
        keyword_candidates.setdefault(phrase, 0)
        word_list = separate_words(phrase, 0)
        candidate_score = 0
        for word in word_list:
            candidate_score += word_score[word]
        keyword_candidates[phrase] = candidate_score
    return keyword_candidates


class Rake(object):
    def __init__(self, stop_words_path, min_char_length=1, max_words_length=5, min_keyword_frequency=1):
        self.__stop_words_path = stop_words_path
        self.__stop_words_pattern = build_stop_word_regex(stop_words_path)
        self.__min_char_length = min_char_length
        self.__max_words_length = max_words_length
        self.__min_keyword_frequency = min_keyword_frequency

    def run(self, text):
        sentence_list = split_sentences(text)

        phrase_list = generate_candidate_keywords(sentence_list, self.__stop_words_pattern, self.__min_char_length, self.__max_words_length)

        word_scores = calculate_word_scores(phrase_list)

        keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores, self.__min_keyword_frequency)

        sorted_keywords = sorted(six.iteritems(keyword_candidates), key=operator.itemgetter(1), reverse=True)
        return sorted_keywords


debug=True
for index, row in train_data.sample(2).iterrows():
    print('____input',row["title"], row["content"] ,row['tags']   )
    print('___content',stripTagsAndUris(row['content']))
    sentenceList = split_sentences(row['title']+'. '+stripTagsAndUris(row['content']))
    #stoppath = "FoxStoplist.txt" #Fox stoplist contains "numbers", so it will not find "natural numbers" like in Table 1.1
    stoppath = "SmartStoplist.txt"  #SMART stoplist misses some of the lower-scoring keywords in Figure 1.5, which means that the top 1/3 cuts off one of the 4.0 score words in Table 1.1
    #stopwordpattern = build_stop_word_regex(stoppath)
    stopwordpattern= """re.compile(r"\ba(?![\w-])|\ba's(?![\w-])|\bable(?![\w-])|\babout(?![\w-])|\babove(?![\w-])|\baccording(?![\w-])|\baccordingly(?![\w-])|\bacross(?![\w-])|\bactually(?![\w-])|\bafter(?![\w-])|\bafterwards(?![\w-])|\bagain(?![\w-])|\bagainst(?![\w-])|\bain't(?![\w-])|\ball(?![\w-])|\ballow(?![\w-])|\ballows(?![\w-])|\balmost(?![\w-])|\balone(?![\w-])|\balong(?![\w-])|\balready(?![\w-])|\balso(?![\w-])|\balthough(?![\w-])|\balways(?![\w-])|\bam(?![\w-])|\bamong(?![\w-])|\bamongst(?![\w-])|\ban(?![\w-])|\band(?![\w-])|\banother(?![\w-])|\bany(?![\w-])|\banybody(?![\w-])|\banyhow(?![\w-])|\banyone(?![\w-])|\banything(?![\w-])|\banyway(?![\w-])|\banyways(?![\w-])|\banywhere(?![\w-])|\bapart(?![\w-])|\bappear(?![\w-])|\bappreciate(?![\w-])|\bappropriate(?![\w-])|\bare(?![\w-])|\baren't(?![\w-])|\baround(?![\w-])|\bas(?![\w-])|\baside(?![\w-])|\bask(?![\w-])|\basking(?![\w-])|\bassociated(?![\w-])|\bat(?![\w-])|\bavailable(?![\w-])|\baway(?![\w-])|\bawfully(?![\w-])|\bb(?![\w-])|\bbe(?![\w-])|\bbecame(?![\w-])|\bbecause(?![\w-])|\bbecome(?![\w-])|\bbecomes(?![\w-])|\bbecoming(?![\w-])|\bbeen(?![\w-])|\bbefore(?![\w-])|\bbeforehand(?![\w-])|\bbehind(?![\w-])|\bbeing(?![\w-])|\bbelieve(?![\w-])|\bbelow(?![\w-])|\bbeside(?![\w-])|\bbesides(?![\w-])|\bbest(?![\w-])|\bbetter(?![\w-])|\bbetween(?![\w-])|\bbeyond(?![\w-])|\bboth(?![\w-])|\bbrief(?![\w-])|\bbut(?![\w-])|\bby(?![\w-])|\bc(?![\w-])|\bc'mon(?![\w-])|\bc's(?![\w-])|\bcame(?![\w-])|\bcan(?![\w-])|\bcan't(?![\w-])|\bcannot(?![\w-])|\bcant(?![\w-])|\bcause(?![\w-])|\bcauses(?![\w-])|\bcertain(?![\w-])|\bcertainly(?![\w-])|\bchanges(?![\w-])|\bclearly(?![\w-])|\bco(?![\w-])|\bcom(?![\w-])|\bcome(?![\w-])|\bcomes(?![\w-])|\bconcerning(?![\w-])|\bconsequently(?![\w-])|\bconsider(?![\w-])|\bconsidering(?![\w-])|\bcontain(?![\w-])|\bcontaining(?![\w-])|\bcontains(?![\w-])|\bcorresponding(?![\w-])|\bcould(?![\w-])|\bcouldn't(?![\w-])|\bcourse(?![\w-])|\bcurrently(?![\w-])|\bd(?![\w-])|\bdefinitely(?![\w-])|\bdescribed(?![\w-])|\bdespite(?![\w-])|\bdid(?![\w-])|\bdidn't(?![\w-])|\bdifferent(?![\w-])|\bdo(?![\w-])|\bdoes(?![\w-])|\bdoesn't(?![\w-])|\bdoing(?![\w-])|\bdon't(?![\w-])|\bdone(?![\w-])|\bdown(?![\w-])|\bdownwards(?![\w-])|\bduring(?![\w-])|\be(?![\w-])|\beach(?![\w-])|\bedu(?![\w-])|\beg(?![\w-])|\beight(?![\w-])|\beither(?![\w-])|\belse(?![\w-])|\belsewhere(?![\w-])|\benough(?![\w-])|\bentirely(?![\w-])|\bespecially(?![\w-])|\bet(?![\w-])|\betc(?![\w-])|\beven(?![\w-])|\bever(?![\w-])|\bevery(?![\w-])|\beverybody(?![\w-])|\beveryone(?![\w-])|\beverything(?![\w-])|\beverywhere(?![\w-])|\bex(?![\w-])|\bexactly(?![\w-])|\bexample(?![\w-])|\bexcept(?![\w-])|\bf(?![\w-])|\bfar(?![\w-])|\bfew(?![\w-])|\bfifth(?![\w-])|\bfirst(?![\w-])|\bfive(?![\w-])|\bfollowed(?![\w-])|\bfollowing(?![\w-])|\bfollows(?![\w-])|\bfor(?![\w-])|\bformer(?![\w-])|\bformerly(?![\w-])|\bforth(?![\w-])|\bfour(?![\w-])|\bfrom(?![\w-])|\bfurther(?![\w-])|\bfurthermore(?![\w-])|\bg(?![\w-])|\bget(?![\w-])|\bgets(?![\w-])|\bgetting(?![\w-])|\bgiven(?![\w-])|\bgives(?![\w-])|\bgo(?![\w-])|\bgoes(?![\w-])|\bgoing(?![\w-])|\bgone(?![\w-])|\bgot(?![\w-])|\bgotten(?![\w-])|\bgreetings(?![\w-])|\bh(?![\w-])|\bhad(?![\w-])|\bhadn't(?![\w-])|\bhappens(?![\w-])|\bhardly(?![\w-])|\bhas(?![\w-])|\bhasn't(?![\w-])|\bhave(?![\w-])|\bhaven't(?![\w-])|\bhaving(?![\w-])|\bhe(?![\w-])|\bhe's(?![\w-])|\bhello(?![\w-])|\bhelp(?![\w-])|\bhence(?![\w-])|\bher(?![\w-])|\bhere(?![\w-])|\bhere's(?![\w-])|\bhereafter(?![\w-])|\bhereby(?![\w-])|\bherein(?![\w-])|\bhereupon(?![\w-])|\bhers(?![\w-])|\bherself(?![\w-])|\bhi(?![\w-])|\bhim(?![\w-])|\bhimself(?![\w-])|\bhis(?![\w-])|\bhither(?![\w-])|\bhopefully(?![\w-])|\bhow(?![\w-])|\bhowbeit(?![\w-])|\bhowever(?![\w-])|\bi(?![\w-])|\bi'd(?![\w-])|\bi'll(?![\w-])|\bi'm(?![\w-])|\bi've(?![\w-])|\bie(?![\w-])|\bif(?![\w-])|\bignored(?![\w-])|\bimmediate(?![\w-])|\bin(?![\w-])|\binasmuch(?![\w-])|\binc(?![\w-])|\bindeed(?![\w-])|\bindicate(?![\w-])|\bindicated(?![\w-])|\bindicates(?![\w-])|\binner(?![\w-])|\binsofar(?![\w-])|\binstead(?![\w-])|\binto(?![\w-])|\binward(?![\w-])|\bis(?![\w-])|\bisn't(?![\w-])|\bit(?![\w-])|\bit'd(?![\w-])|\bit'll(?![\w-])|\bit's(?![\w-])|\bits(?![\w-])|\bitself(?![\w-])|\bj(?![\w-])|\bjust(?![\w-])|\bk(?![\w-])|\bkeep(?![\w-])|\bkeeps(?![\w-])|\bkept(?![\w-])|\bknow(?![\w-])|\bknows(?![\w-])|\bknown(?![\w-])|\bl(?![\w-])|\blast(?![\w-])|\blately(?![\w-])|\blater(?![\w-])|\blatter(?![\w-])|\blatterly(?![\w-])|\bleast(?![\w-])|\bless(?![\w-])|\blest(?![\w-])|\blet(?![\w-])|\blet's(?![\w-])|\blike(?![\w-])|\bliked(?![\w-])|\blikely(?![\w-])|\blittle(?![\w-])|\blook(?![\w-])|\blooking(?![\w-])|\blooks(?![\w-])|\bltd(?![\w-])|\bm(?![\w-])|\bmainly(?![\w-])|\bmany(?![\w-])|\bmay(?![\w-])|\bmaybe(?![\w-])|\bme(?![\w-])|\bmean(?![\w-])|\bmeanwhile(?![\w-])|\bmerely(?![\w-])|\bmight(?![\w-])|\bmore(?![\w-])|\bmoreover(?![\w-])|\bmost(?![\w-])|\bmostly(?![\w-])|\bmuch(?![\w-])|\bmust(?![\w-])|\bmy(?![\w-])|\bmyself(?![\w-])|\bn(?![\w-])|\bname(?![\w-])|\bnamely(?![\w-])|\bnd(?![\w-])|\bnear(?![\w-])|\bnearly(?![\w-])|\bnecessary(?![\w-])|\bneed(?![\w-])|\bneeds(?![\w-])|\bneither(?![\w-])|\bnever(?![\w-])|\bnevertheless(?![\w-])|\bnew(?![\w-])|\bnext(?![\w-])|\bnine(?![\w-])|\bno(?![\w-])|\bnobody(?![\w-])|\bnon(?![\w-])|\bnone(?![\w-])|\bnoone(?![\w-])|\bnor(?![\w-])|\bnormally(?![\w-])|\bnot(?![\w-])|\bnothing(?![\w-])|\bnovel(?![\w-])|\bnow(?![\w-])|\bnowhere(?![\w-])|\bo(?![\w-])|\bobviously(?![\w-])|\bof(?![\w-])|\boff(?![\w-])|\boften(?![\w-])|\boh(?![\w-])|\bok(?![\w-])|\bokay(?![\w-])|\bold(?![\w-])|\bon(?![\w-])|\bonce(?![\w-])|\bone(?![\w-])|\bones(?![\w-])|\bonly(?![\w-])|\bonto(?![\w-])|\bor(?![\w-])|\bother(?![\w-])|\bothers(?![\w-])|\botherwise(?![\w-])|\bought(?![\w-])|\bour(?![\w-])|\bours(?![\w-])|\bourselves(?![\w-])|\bout(?![\w-])|\boutside(?![\w-])|\bover(?![\w-])|\boverall(?![\w-])|\bown(?![\w-])|\bp(?![\w-])|\bparticular(?![\w-])|\bparticularly(?![\w-])|\bper(?![\w-])|\bperhaps(?![\w-])|\bplaced(?![\w-])|\bplease(?![\w-])|\bplus(?![\w-])|\bpossible(?![\w-])|\bpresumably(?![\w-])|\bprobably(?![\w-])|\bprovides(?![\w-])|\bq(?![\w-])|\bque(?![\w-])|\bquite(?![\w-])|\bqv(?![\w-])|\br(?![\w-])|\brather(?![\w-])|\brd(?![\w-])|\bre(?![\w-])|\breally(?![\w-])|\breasonably(?![\w-])|\bregarding(?![\w-])|\bregardless(?![\w-])|\bregards(?![\w-])|\brelatively(?![\w-])|\brespectively(?![\w-])|\bright(?![\w-])|\bs(?![\w-])|\bsaid(?![\w-])|\bsame(?![\w-])|\bsaw(?![\w-])|\bsay(?![\w-])|\bsaying(?![\w-])|\bsays(?![\w-])|\bsecond(?![\w-])|\bsecondly(?![\w-])|\bsee(?![\w-])|\bseeing(?![\w-])|\bseem(?![\w-])|\bseemed(?![\w-])|\bseeming(?![\w-])|\bseems(?![\w-])|\bseen(?![\w-])|\bself(?![\w-])|\bselves(?![\w-])|\bsensible(?![\w-])|\bsent(?![\w-])|\bserious(?![\w-])|\bseriously(?![\w-])|\bseven(?![\w-])|\bseveral(?![\w-])|\bshall(?![\w-])|\bshe(?![\w-])|\bshould(?![\w-])|\bshouldn't(?![\w-])|\bsince(?![\w-])|\bsix(?![\w-])|\bso(?![\w-])|\bsome(?![\w-])|\bsomebody(?![\w-])|\bsomehow(?![\w-])|\bsomeone(?![\w-])|\bsomething(?![\w-])|\bsometime(?![\w-])|\bsometimes(?![\w-])|\bsomewhat(?![\w-])|\bsomewhere(?![\w-])|\bsoon(?![\w-])|\bsorry(?![\w-])|\bspecified(?![\w-])|\bspecify(?![\w-])|\bspecifying(?![\w-])|\bstill(?![\w-])|\bsub(?![\w-])|\bsuch(?![\w-])|\bsup(?![\w-])|\bsure(?![\w-])|\bt(?![\w-])|\bt's(?![\w-])|\btake(?![\w-])|\btaken(?![\w-])|\btell(?![\w-])|\btends(?![\w-])|\bth(?![\w-])|\bthan(?![\w-])|\bthank(?![\w-])|\bthanks(?![\w-])|\bthanx(?![\w-])|\bthat(?![\w-])|\bthat's(?![\w-])|\bthats(?![\w-])|\bthe(?![\w-])|\btheir(?![\w-])|\btheirs(?![\w-])|\bthem(?![\w-])|\bthemselves(?![\w-])|\bthen(?![\w-])|\bthence(?![\w-])|\bthere(?![\w-])|\bthere's(?![\w-])|\bthereafter(?![\w-])|\bthereby(?![\w-])|\btherefore(?![\w-])|\btherein(?![\w-])|\btheres(?![\w-])|\bthereupon(?![\w-])|\bthese(?![\w-])|\bthey(?![\w-])|\bthey'd(?![\w-])|\bthey'll(?![\w-])|\bthey're(?![\w-])|\bthey've(?![\w-])|\bthink(?![\w-])|\bthird(?![\w-])|\bthis(?![\w-])|\bthorough(?![\w-])|\bthoroughly(?![\w-])|\bthose(?![\w-])|\bthough(?![\w-])|\bthree(?![\w-])|\bthrough(?![\w-])|\bthroughout(?![\w-])|\bthru(?![\w-])|\bthus(?![\w-])|\bto(?![\w-])|\btogether(?![\w-])|\btoo(?![\w-])|\btook(?![\w-])|\btoward(?![\w-])|\btowards(?![\w-])|\btried(?![\w-])|\btries(?![\w-])|\btruly(?![\w-])|\btry(?![\w-])|\btrying(?![\w-])|\btwice(?![\w-])|\btwo(?![\w-])|\bu(?![\w-])|\bun(?![\w-])|\bunder(?![\w-])|\bunfortunately(?![\w-])|\bunless(?![\w-])|\bunlikely(?![\w-])|\buntil(?![\w-])|\bunto(?![\w-])|\bup(?![\w-])|\bupon(?![\w-])|\bus(?![\w-])|\buse(?![\w-])|\bused(?![\w-])|\buseful(?![\w-])|\buses(?![\w-])|\busing(?![\w-])|\busually(?![\w-])|\buucp(?![\w-])|\bv(?![\w-])|\bvalue(?![\w-])|\bvarious(?![\w-])|\bvery(?![\w-])|\bvia(?![\w-])|\bviz(?![\w-])|\bvs(?![\w-])|\bw(?![\w-])|\bwant(?![\w-])|\bwants(?![\w-])|\bwas(?![\w-])|\bwasn't(?![\w-])|\bway(?![\w-])|\bwe(?![\w-])|\bwe'd(?![\w-])|\bwe'll(?![\w-])|\bwe're(?![\w-])|\bwe've(?![\w-])|\bwelcome(?![\w-])|\bwell(?![\w-])|\bwent(?![\w-])|\bwere(?![\w-])|\bweren't(?![\w-])|\bwhat(?![\w-])|\bwhat's(?![\w-])|\bwhatever(?![\w-])|\bwhen(?![\w-])|\bwhence(?![\w-])|\bwhenever(?![\w-])|\bwhere(?![\w-])|\bwhere's(?![\w-])|\bwhereafter(?![\w-])|\bwhereas(?![\w-])|\bwhereby(?![\w-])|\bwherein(?![\w-])|\bwhereupon(?![\w-])|\bwherever(?![\w-])|\bwhether(?![\w-])|\bwhich(?![\w-])|\bwhile(?![\w-])|\bwhither(?![\w-])|\bwho(?![\w-])|\bwho's(?![\w-])|\bwhoever(?![\w-])|\bwhole(?![\w-])|\bwhom(?![\w-])|\bwhose(?![\w-])|\bwhy(?![\w-])|\bwill(?![\w-])|\bwilling(?![\w-])|\bwish(?![\w-])|\bwith(?![\w-])|\bwithin(?![\w-])|\bwithout(?![\w-])|\bwon't(?![\w-])|\bwonder(?![\w-])|\bwould(?![\w-])|\bwould(?![\w-])|\bwouldn't(?![\w-])|\bx(?![\w-])|\by(?![\w-])|\byes(?![\w-])|\byet(?![\w-])|\byou(?![\w-])|\byou'd(?![\w-])|\byou'll(?![\w-])|\byou're(?![\w-])|\byou've(?![\w-])|\byour(?![\w-])|\byours(?![\w-])|\byourself(?![\w-])|\byourselves(?![\w-])|\bz(?![\w-])|\bzero(?![\w-])",
re.IGNORECASE|re.UNICODE)"""

    #stopwordpattern = build_stop_word_regex(stoppath)

    # generate candidate keywords
    phraseList = generate_candidate_keywords(sentenceList, stopwordpattern)
    if debug: print('____prhas',phraseList)


    # calculate individual word scores
    wordscores = calculate_word_scores(phraseList)
    if debug: print('____words',wordscores)


    # generate candidate keyword scores
    keywordcandidates = generate_candidate_keyword_scores(phraseList, wordscores)
    if debug: print('____keyw',keywordcandidates)

    sortedKeywords = sorted(six.iteritems(keywordcandidates), key=operator.itemgetter(1), reverse=True)
    if debug: print('____sort',sortedKeywords)

    totalKeywords = len(sortedKeywords)
    if debug: print('____ total',totalKeywords)
    print('____result',sortedKeywords[0:(totalKeywords // 3)])

    #rake = Rake("SmartStoplist.txt")
    #keywords = rake.run(text)
    #print(keywords)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, ngram_range=(1,2),stop_words='english')

sample=train_data.sample(3)
for index, row in sample.iterrows():
    print('____input',row["title"], row["content"] ,'<h1>',row['tags'],'</h1>'   )
    data=row['title']+'. '+stripTagsAndUris(row['content'])
    sample[index,'content']=data
    
tfidf = tfidf_vectorizer.fit_transform(sample['content'])
print(data_samples)

print(tfidf_vectorizer.vocabulary_)
words=pd.DataFrame.from_dict(tfidf_vectorizer.vocabulary_,orient='index')
print(words.sort_values(by=0,ascending=[0] ))
#print(tfidf_vectorizer.idf_)
#tfinv= tfidf_vectorizer.inverse_transform(tfidf)
#print(tfinv)